*   Name: Brinda Gupta
*   Enrollment Number: 19/11/EC/001
*   Branch: CSE
*   Email: brindagupta2002@gmail.com

##CSV Function

In [103]:
import csv

def create_empty_csv(filename, headers):
  """
  Creates an empty CSV file with specified headers.

  Args:
      filename: The name of the CSV file to create.
      headers: A list of strings representing the CSV header row.
  """
  with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)

# Example usage
headers = ["Fault Percentage of Single Layer", "Inference accuracy of Single Layer", "Fault Percentage of Triple Layer", "Inference accuracy of Triple Layer"]
create_empty_csv("/content/drive/MyDrive/MTech/Experiment/Inference.csv", headers)

In [104]:
import csv
def format_floats(row, precision=5):
    return [f"{x:.{precision}f}" if isinstance(x, float) else x for x in row]
def write_to_csv(file_path, row, precision=5):
    formatted_row = format_floats(row, precision)
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(formatted_row)

In [105]:
import torch
import torch.nn as nn
import torchvision
from pathlib import Path

#Creating Masked Tensor Function

In [106]:
def create_masked_tensor(size=(784, 784), percentage=10):


  # Create a base tensor with all values set to -1
  tensor = torch.ones(size) * -1

  # Calculate the number of elements to mask (10% for both 0 and 1)
  num_elements = tensor.numel()  # Total number of elements in the tensor
  num_to_mask_zero = int(0.85 * percentage/100 * num_elements)
  num_to_mask_one = int(0.22 * percentage/100 * num_elements)

  # Create random masks for 0 and 1 (size matches the tensor)
  mask_zero = torch.rand(size).lt(num_to_mask_zero / num_elements).float()
  mask_one = torch.rand(size).lt(num_to_mask_one / num_elements).float()

  # Apply masks to set elements to 0 or 1
  tensor = tensor * (1 - mask_zero - mask_one) + mask_zero * 0 + mask_one * 1

  return tensor

#Creating Weight Altering Function for Single Layer Memristor Crossbar

In [107]:
def alter_weights_single_layer(model, tensor1, tensor2):
    # Ensure the tensor shapes match the corresponding weights
    assert tensor1.shape == (784, 784), "tensor1 shape must be (784, 784)"
    assert tensor2.shape == (10, 784), "tensor2 shape must be (10, 784)"

    # Get the current state dictionary of the model
    state_dict = model.state_dict()

    # Alter weights for the first layer
    weights1 = state_dict['layer1.weight'].clone()
    mask1_pos = (tensor1 == 1)
    mask1_zero = (tensor1 == 0)
    weights1[mask1_pos] = 1
    weights1[mask1_zero] = 0
    state_dict['layer1.weight'] = weights1

    # Alter weights for the second layer
    weights2 = state_dict['layer2.weight'].clone()
    mask2_pos = (tensor2 == 1)
    mask2_zero = (tensor2 == 0)
    weights2[mask2_pos] = 1
    weights2[mask2_zero] = 0
    state_dict['layer2.weight'] = weights2

    # Load the altered weights back into the model
    model.load_state_dict(state_dict)

    return model

#Creating Weight Altering Function for Triple Layer Memristor Crossbar

In [108]:
def alter_weights_triple_layers(model, tensorlist):

    # Get the current state dictionary of the model
    state_dict = model.state_dict()

    # Alter weights for the first layer
    weights11 = state_dict['layer1.weight'].clone()
    mask1_pos = (tensorlist[0] == 1)
    mask1_zero = (tensorlist[0] == 0)
    weights11[mask1_pos] = 1
    weights11[mask1_zero] = 0

    weights12 = state_dict['layer1.weight'].clone()
    mask1_pos = (tensorlist[2] == 1)
    mask1_zero = (tensorlist[2] == 0)
    weights12[mask1_pos] = 1
    weights12[mask1_zero] = 0

    weights13 = state_dict['layer1.weight'].clone()
    mask1_pos = (tensorlist[4] == 1)
    mask1_zero = (tensorlist[4] == 0)
    weights13[mask1_pos] = 1
    weights13[mask1_zero] = 0

    state_dict['layer1.weight'] = (weights11+weights12+weights13)/3

    # Alter weights for the second layer
    # Alter weights for the first layer
    weights21 = state_dict['layer2.weight'].clone()
    mask2_pos = (tensorlist[1] == 1)
    mask2_zero = (tensorlist[1] == 0)
    weights21[mask2_pos] = 1
    weights21[mask2_zero] = 0

    weights22 = state_dict['layer2.weight'].clone()
    mask2_pos = (tensorlist[3] == 1)
    mask2_zero = (tensorlist[3] == 0)
    weights22[mask2_pos] = 1
    weights22[mask2_zero] = 0

    weights23 = state_dict['layer2.weight'].clone()
    mask2_pos = (tensorlist[5] == 1)
    mask2_zero = (tensorlist[5] == 0)
    weights23[mask2_pos] = 1
    weights23[mask2_zero] = 0

    state_dict['layer2.weight'] = (weights21+weights22+weights23)/3

    # Load the altered weights back into the model
    model.load_state_dict(state_dict)

    return model

#Calculate of percentages

In [109]:
def calculate_value_percentages(tensor):
  """
  Calculates the percentage of 0s and 1s in a PyTorch tensor.

  Args:
      tensor: A PyTorch tensor.

  Returns:
      A dictionary containing the percentage of 0s and 1s.
  """
  total_elements = torch.numel(tensor)

  # Count occurrences of 0 and 1
  count_0 = torch.count_nonzero(tensor == 0).item()
  count_1 = torch.count_nonzero(tensor == 1).item()

  # Calculate percentages
  percent_0 = (count_0 / total_elements) * 100
  percent_1 = (count_1 / total_elements) * 100

  return {"0": percent_0, "1": percent_1}

#Model

In [110]:
#Model
class Baseline(nn.Module):
  def __init__(self):
      super().__init__()
      self.layer1 = nn.Linear(784, 784)
      self.act1 = nn.ReLU()
      self.layer2 = nn.Linear(784, 10)

  def forward(self, x):
      x = self.act1(self.layer1(x))
      x = self.layer2(x)
      return x

#Main

In [111]:
for fault_percentage in range(0, 15, 2):
  #Downloading MNIST Test Data
  test = torchvision.datasets.MNIST('data', train=False, download=True)
  len(test)
  X_test = test.data.reshape(-1, 784).float() / 255.0
  y_test = test.targets


  #Downloading Weights
  MODEL_PATH = Path("/content/drive/MyDrive/MTech/Experiment")
  MODEL_NAME = "base_mnist_clamped.pth"
  MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
  # Loading the saved model
  model = Baseline()
  model.load_state_dict(torch.load(MODEL_SAVE_PATH))

  #Making Masks for Single-Layer Memristor Crossbar
  Layer1_mask = create_masked_tensor(size=(784, 784), percentage=fault_percentage)
  Layer2_mask = create_masked_tensor(size=(10, 784), percentage=fault_percentage)

  #Altering the Weights for Single Layer Memristor crossbar
  model = alter_weights_single_layer(model, Layer1_mask, Layer2_mask)

  #Percentage of total faults in Single Layer Memristor crossbar
  percentages = calculate_value_percentages(Layer1_mask)
  percentage1 = 0
  percentage1 += ((percentages['0']+percentages['1'])*7.84*784)
  percentages = calculate_value_percentages(Layer2_mask)
  percentage1 += ((percentages['0']+percentages['1'])*7.84*10)
  percentage1 /= (784*794)
  percentage1 *= 100
  # print("Percentage of faults in single layer:", percentage1)

  #Testing Accuracy of a single layer Memristor crossbar-based Neural Network
  loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), shuffle=True, batch_size=100)
  accuracy1 = 0
  model.eval()
  with torch.no_grad():
      for X_batch, y_batch in loader:
          y_pred = model(X_batch)
          batch_acc = (torch.argmax(y_pred, 1) == y_batch).float().mean()
          accuracy1 += batch_acc.item() * X_batch.size(0)
  # Calculate overall accuracy
  accuracy1 /= len(loader.dataset)
  accuracy1 *= 100
  # print("Model accuracy with", i, "percent faults on test set: %.5f%%" % accuracy1)



  #Downloading MNIST Test Data
  test = torchvision.datasets.MNIST('data', train=False, download=True)
  len(test)
  X_test = test.data.reshape(-1, 784).float() / 255.0
  y_test = test.targets

  #Dowloading Weights again
  MODEL_PATH = Path("/content/drive/MyDrive/MTech/Experiment")
  MODEL_NAME = "base_mnist_clamped.pth"
  MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
  # Loading the saved model
  model = Baseline()
  model.load_state_dict(torch.load(MODEL_SAVE_PATH))

  #Getting list for the fault-mask of three layers of memristor crossbar
  Tensor_list=[]
  for j in range(3):
      Layer1_mask = create_masked_tensor(size=(784, 784), percentage=fault_percentage)
      Layer2_mask = create_masked_tensor(size=(10, 784), percentage=fault_percentage)
      Tensor_list.extend([Layer1_mask, Layer2_mask])
  model = alter_weights_triple_layers(model, Tensor_list)


  #Percentage of total faults in Triple-layer Memristor crossbar
  percentage3 = 0
  for i in range(3):
    percentages = calculate_value_percentages(Tensor_list[i*2+0])
    percentage3+=((percentages['0']+percentages['1'])*7.84*784)
    percentages = calculate_value_percentages(Tensor_list[i*2+1])
    percentage3+=((percentages['0']+percentages['1'])*7.84*10)
  percentage3 = percentage3*100/(794*784)/3
  # print("Percentage of faults in triple layers:", percentage3)

  #Testing Accuracy of a triple layer Memristor crossbar-based Neural Network
  loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), shuffle=True, batch_size=100)
  accuracy3 = 0
  model.eval()
  with torch.no_grad():
      for X_batch, y_batch in loader:
          y_pred = model(X_batch)
          batch_acc = (torch.argmax(y_pred, 1) == y_batch).float().mean()
          accuracy3 += batch_acc.item() * X_batch.size(0)
  # Calculate overall accuracy
  accuracy3 /= len(loader.dataset)
  accuracy3 *= 100
  # print("Model accuracy on test set: %.5f%%" % accuracy3)
  # if fault_percentage % 2 == 0:
  write_to_csv("/content/drive/MyDrive/MTech/Experiment/Inference.csv", [percentage1, accuracy1, percentage3, accuracy3])
